In [1]:
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from load_data import load_data
df_cut = 2000
df_train, df_valid = load_data(df_cut)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yaros\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yaros\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\yaros\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yaros\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df_train.shape

(2000, 5)

In [5]:
df_train.head()

,Title,Body,Tags,text,clean_text
42467,How to know what company protect a website?,"<p><a href=""https://www.genecards.org"" rel=""no...",<web-scraping>,How to know what company protect a website? <p...,know company protect website http www genecard...
6370,Macro to find cells with value and replacing w...,I want to search for all cells in a column tha...,<excel><vba>,Macro to find cells with value and replacing w...,macro find cell value replacing value adjacent...
14248,closing previous jframes while opening another,i have 2 jframes f1&f2 both having buttons b1&...,<java><swing><jframe>,closing previous jframes while opening another...,closing previous jframes opening another jfram...
15077,Writing Apache Zookeeper like service in go,<p>I want to write a very simple (but fully fu...,<go><apache-zookeeper>,Writing Apache Zookeeper like service in go <p...,writing apache zookeeper like service want wri...
28634,Go:get fragment values in url,sample code \r\n\r\n<!-- begin snippet: js hid...,<go><httphandler>,Go:get fragment values in url sample code \r\n...,get fragment value url sample code func main f...


In [6]:
n_topics = 7
n_iter = 18

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.95, min_df=5, stop_words='english')
X = vectorizer.fit_transform(df_train['clean_text'])
# X_valid


In [8]:
X.shape

(2000, 2035)

In [9]:
import time

In [10]:
from sklearn.decomposition import LatentDirichletAllocation

start_train_sk = time.time()
lda = LatentDirichletAllocation(learning_method='batch',
                                n_components=n_topics,
                                max_iter = n_iter,
                                n_jobs=None,
                                random_state=42)
lda.fit(X)
end_train_sk = time.time()

def print_topics(model, vectorizer, top_n=10):
    words = vectorizer.get_feature_names_out()
    for idx, topic in enumerate(model.components_):
        print(f"\nТема #{idx+1}:")
        print(" ".join([words[i] for i in topic.argsort()[:-top_n - 1:-1]]))
start_pred_sk = time.time()
print_topics(lda, vectorizer)
end_pred_sk = time.time()



Тема #1:
public new string class void private user http return import

Тема #2:
android java layout com app view content lang org foo

Тема #3:
int function return input amp include cout std react module

Тема #4:
string array value list number int return code new print

Тема #5:
like use want using time text way need color code

Тема #6:
file error line python command using version code user run

Тема #7:
image file data div php row html column value button


In [11]:
from lda import LDA
tokenized_docs = [doc.split() for doc in df_train['clean_text']]

start_train_cu = time.time()
model = LDA(n_topics=n_topics, alpha=0.1, beta=0.1, n_iter=n_iter)
model.fit(tokenized_docs)
end_train_cu = time.time()


# Топ слова по темам
start_pred_cu = time.time()
for idx, topic in enumerate(model.get_topic_words(top_n=8)):
    print(f"Тема #{idx+1}: {' '.join(topic)}")
end_pred_cu = time.time()


Training LDA: 100%|██████████| 18/18 [04:53<00:00, 16.32s/it]

Тема #1: string public name new return class null int
Тема #2: android class layout image com width app height
Тема #3: value input name function var type get php
Тема #4: int array number return list code time print
Тема #5: java new add system public class void string
Тема #6: file data want like use name using value
Тема #7: file error app module http using version com


In [12]:
print(f"[sklearn] Время обучения: {end_train_sk - start_train_sk:.2f} сек")
print(f"[sklearn] Время предсказания: {end_pred_sk - start_pred_sk:.2f} сек")
print(f"[LDA_custom] Время обучения: {end_train_cu - start_train_cu:.2f} сек")
print(f"[LDA_custom] Время предсказания: {end_pred_cu - start_pred_cu:.2f} сек")

[sklearn] Время обучения: 12.68 сек
[sklearn] Время предсказания: 0.00 сек
[LDA_custom] Время обучения: 294.20 сек
[LDA_custom] Время предсказания: 0.00 сек


In [13]:
from gensim.models import CoherenceModel
from gensim import corpora

def compute_coherence_gensim(topics_words, tokenized_texts, coherence='c_v'):
    dictionary = corpora.Dictionary(tokenized_texts)
    coherence_model = CoherenceModel(topics=topics_words,
                                     texts=tokenized_texts,
                                     dictionary=dictionary,
                                     coherence=coherence)
    return coherence_model.get_coherence()


In [14]:
tokenized_docs = [doc.split() for doc in df_train['clean_text']]
top_words_lda = model.get_topic_words(top_n=10)

coherence_lda = compute_coherence_gensim(top_words_lda, tokenized_docs)
print(f"Когерентность custom реализации LDA: {coherence_lda:.4f}")


Когерентность custom реализации LDA: 0.5854


In [15]:
def get_sklearn_topics(lda_model, vectorizer, top_n=10):
    feature_names = vectorizer.get_feature_names_out()
    topics = []
    for topic in lda_model.components_:
        top_indices = topic.argsort()[:-top_n - 1:-1]
        topics.append([feature_names[i] for i in top_indices])
    return topics

top_words_sklearn = get_sklearn_topics(lda, vectorizer, top_n=10)

coherence_sklearn = compute_coherence_gensim(top_words_sklearn, tokenized_docs)
print(f"Когерентность sklearn LDA: {coherence_sklearn:.4f}")


Когерентность sklearn LDA: 0.5367
